<a href="https://colab.research.google.com/github/gonzalobarrazueta/TF-20181e098-20191a972-201920156/blob/main/MAPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Imports**

In [ ]:
# Perlin Noise
!pip install perlin_noise
import matplotlib.pyplot as plt
import numpy as np
from perlin_noise import PerlinNoise
# ------------
import csv
import json
# Sección Aristas
from datetime import datetime
from math import radians, cos, sin, asin, sqrt
import random
# Dijkstra/Priority Queue
import heapq as hq
import math
# ------------
import numpy as np
import graphviz as gv #sirve para dibujar grafos

##**Diccionario de calles**
Crea un diccionario donde cada nombre de calle ***(key)*** tiene asignado un `id` ***(value)***

In [7]:
def addKey_Values(dictionary, key, value):
    dictionary[key] = value

In [8]:
def printDictionary(dictionary):
    for k, v in dictionary.items():
        print(f"{k}: {v}")

In [9]:
def cargarDatos(dictionary):
    with open('Calles de San Martín de Porres.csv') as file: 
        reader = csv.reader(file)
        
        for id, row in enumerate(reader):
            addKey_Values(dictionary, row[0].lower(), id)

In [10]:
def createStreetsDictionary():   
    streets = {}
    cargarDatos(streets)
    return streets

In [11]:
streetsDictionary = createStreetsDictionary()

##**Lista de Adyacencia de calles y sus intersecciones**




In [12]:
def printAdList(graph): 
    for i, intersection in enumerate(graph):
        print(i, intersection)
    
    print(f"Cantidad de vértices = {len(graph)}")

In [13]:
def streetExists(streetName):
    if streetName in streetsDictionary:
        id = streetsDictionary[streetName]
    else:
        id = -1 # la calle no está mapeada
    return id

In [14]:
# adjacencyList: arreglo de arreglos, donde la pos del primer elemento (1er arreglo)
# es la calle y el arreglo son todas las calles con las que se intersecta
def createAdjacencyList(adjacencyList):
    with open('Calles de San Martín de Porres.csv') as file: 
        reader = csv.reader(file)
        for row in reader:  
            intersections = []
            for i in range(len(row)):
                if i != 0:
                    intersections.append(streetExists(row[i].lower()))
            adjacencyList.append(intersections)

    return adjacencyList

In [15]:
# creal la lista de adyacencia
intersectionsAdList = createAdjacencyList([])

##**Lista de Adyacencia de intersecciones (Grafo Principal)**

In [16]:
def createKeys(graph):
    intersectionsIDs = {}

    for i in range(len(graph)):
        for street in graph[i]:
            # si las keys aún no han sido creadas, las agrega al diccionario
            if (i, street) not in intersectionsIDs and street != -1:
                intersectionsIDs[(i, street)] = []

    return intersectionsIDs

In [17]:
def addIntersections(graph, i, j):  
    street = graph[i][j]

    if street != -1:
        # agrego la intersección már próxima hacia la izquierda
        if j > 0 and graph[i][j - 1] != -1:
            tup = (i, graph[i][j - 1])
            map[(i, street)].append(tup)

        # agrego la intersección már próxima hacia la derecha
        if j < len(graph[i]) - 1 and graph[i][j + 1] != -1:
            tup = (i, graph[i][j + 1])
            map[(i, street)].append(tup)

In [18]:
def addIntersectionsReversed(graph, i, j):  
    street = graph[i][j]

    # ahora voy al reverso de la lista
    if i != -1 and street != -1:
        if (street < len(graph)) and (i in graph[street]):
            tup = (street, i)
            map[(i, street)].append(tup)
            
            k = graph[street].index(i) # k = 2

            if k > 0 and graph[street][k - 1] != -1:
                tup = (street, graph[street][k - 1])
                map[(i, street)].append(tup)
        
            if k < len(graph[street]) - 1 and graph[street][k + 1] != -1:
                tup = (street, graph[street][k + 1])
                map[(i, street)].append(tup)

In [19]:
def joinIntersections(graph):
    n = len(graph)
    for i in range(n):        
        for j in range(len(graph[i])):
            addIntersections(graph, i, j)
            addIntersectionsReversed(graph, i, j)

In [20]:
# calcula el total de nodos solo par el grafo "map"
def totalNodes(graph):
    total = 0 
    for key, value in graph.items():
        total += len(value)
    return total

In [21]:
# grafo principal
map = createKeys(intersectionsAdList)
joinIntersections(intersectionsAdList)

##**Diccionario de Latitudes, Longitudes y `ids` para cada intersección**

In [22]:
def formatearLatitudeorLongitude(input):
    separar = input
    separar = separar.split(".");

    union= separar[0]+"."+separar[1]+separar[2];
    variable_float = float(union)

    return variable_float

In [23]:
# devuelve solo la latitud y longitud, no el id
def getOnlyLatLong(key):
    return (latlong[key][0], latlong[key][1])

In [24]:
def getInterId(key):
    return latlong[key][2]

In [25]:
def createLatLongDictionary(latlong):
    with open("Latitud y Longitud de calles.csv") as file: 
        reader = csv.reader(file)      
        # ignora el header (la 1era fila del csv)
        next(reader)
        # se le asignará un id a cada vértice
        id = 0
        for row in reader:
            # busco el id de la calle en mi diccionario de calles 
            intersection = streetExists(row[0]), streetExists(row[1])
            if intersection in map:
                latlong[intersection] = formatearLatitudeorLongitude(row[2]), formatearLatitudeorLongitude(row[3]), id
                id += 1
    return latlong
     
latlong = createLatLongDictionary({})

##**Agregar valores aleatorios**
Agregar valores aleatorios al grafo principal (`map`). Los valores random fueron generados con Perlin Noise

In [26]:
def getGraphSize(graph):
    return len(graph)

In [27]:
def perlinNoise():
    # tamaño de cada de arreglo
    xpix = 10
    # cantidad de arreglos
    ypix = getGraphSize(map)
    noise = PerlinNoise(octaves=5, seed=1)
    pic = [[noise([i/xpix, j/ypix]) for j in range(xpix)] for i in range(ypix)]

    return np.array(np.array(pic)*100, dtype=int)

randValues = perlinNoise()

In [28]:
def sameNode(n1, n2):
    x1, y1 = n1
    x2, y2 = n2

    if (x1 == y2) and (y1 == x2):
        return True
    return False

In [29]:
def getListRandValues(n):
    r = random.randrange(0, n)
    
    while sum(randValues[r]) == 0:
        r = random.randrange(0, n)

    return list(randValues[r])

In [30]:
def getPositive(num):
    if num < 0:
        return num * -1
    else:
        return num

In [31]:
# agrega un valor random (generado con perlin noise) al diccionario de lat y longs
def addRandValue(dictionary):

    randv = getListRandValues(len(latlong))

    for key, value in map.items():
        l = []
        for u in value:
            if sameNode(key, u):
                u += (0,)      
                l.append(u)
                continue

            if len(randv) > 0:
                u += (getPositive(randv.pop()),)
            else:
                randv = getListRandValues(len(latlong))
                u += (getPositive(randv.pop()),)
            l.append(u)
        dictionary[key] = l
    return dictionary
    
map2 = addRandValue({})

##**Escribir archivos**

In [32]:
def writeNodes():
    with open("nodes.txt", "w") as f:
        with open('Calles de San Martín de Porres.csv') as file: 
            reader = csv.reader(file)

            for row in reader:
                for i in range(1, len(row)):                    
                    f.write(f"{row[0]}, {row[i]}\n")
writeNodes()

In [33]:
# Streets Dictionary
def writeStreetsDictionary():
    with open("streetNames.txt", "w") as f:
        for key, value in streetsDictionary.items():
            f.write(key + "\n")
writeStreetsDictionary()

In [34]:
# Intersections Adjacency List
def writeStreetsAdList():
    with open("streetsAdList.txt", "w") as f:
        for streets in intersectionsAdList:
            for i in range(len(streets)):
                streets[i] = str(streets[i])    
            f.write(', '.join(streets) + "\n")        
writeStreetsAdList()

In [35]:
# Write street map
def writeStreetMap():
    with open("map.txt", "w") as f:  
        for key, value in map.items():
            listWOCommas = (str(value).strip('[]')).replace(",", "")
            f.write(str(key) + " " + listWOCommas + '\n')
writeStreetMap()

##**Aristas**

In [36]:
# devuelve el tiempo actual automáticamente
def getCurrentTime():

    now = datetime.now()
    time = now.strftime("%H %p")
    time = time.split(" ")

    # convierto la hora a la timezone de Perú
    if time[0] <= "5":
        time[0] = 24 - (5 - int(time[0]))
    else:
        time[0] = int(time[0]) - 5
    
    return str(time[0])

def askTime():
    time = int(input("Digite la hora en formato de 24hrs: "));
    if time not in range(0, 24):
        time = askTime() 
    return time

In [37]:
def getlongitude(n1, n2):
    lon1,lat1=n1
    lon2,lat2=n2
    if lon1 == lat2 and lon2 == lat1:
      longitude = 0
    else:
      lon1 = radians(lon1)
      lon2 = radians(lon2)
      lat1 = radians(lat1)
      lat2 = radians(lat2)
      dlon = lon2 - lon1
      dlat = lat2 - lat1
      a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
      c = 2 * asin(sqrt(a)) 
      r = 6371
      longitude = c * r
    
    return longitude

In [38]:
def point_percent(time):
    percent = 0
    # de 0am - 4am
    if time in range(0, 5):
        percent = 0.1
    
    # de 5am - 8am
    if time in range(5, 9):
        percent = 0.9

    # de 9am - 16pm
    if time in range(9, 18):
        percent = 0.5

    # de 18pm - 20pm
    if time in range(18, 21):
        percent = 1.3

    # de 21pm - 23pm
    if time in range(21, 23):
        percent = 0.3

    return percent

In [39]:
def getFactor(time, val):
    # el factor está en el rango de 0 - 50
    # dependiendo de la hora, mi valor va estar en un determinado rango
    # puntos por hora
    # índice: hora, elemento: puntaje
                 # 0  1  2  3  4  5  6   7   8   9   10  11  12  13  14  15  16  17  18  19  20  21  22  23  
    time_points = [0, 1, 1, 1, 2, 3, 15, 19, 17, 14, 13, 10, 10, 11, 12, 13, 14, 16, 17, 20, 19, 15, 4, 2]

    percent = point_percent(time)
    factor = int((val * percent) + time_points[time])

    if factor > 50:
        factor = 50

    return factor

In [40]:
def calculateWeight(n1, n2, time, val):
    
    longitude = getlongitude(n1, n2) * 100
    factor = getFactor(time, val)
    
    weight = longitude * factor

    if longitude > 0:
        weight = round(factor + longitude, 2)
    
    if (weight > 100):
        return 99
    else:
        return int(weight)

In [51]:
# el nuevo grafo será un diccionario, donde cada "key" contiene 1 arreglo de arreglos
# cada arreglo dentro del principal, tiene solo 2 posiciones
# 1era posición del arreglo: 1 tuple de intersecciones
# 2da posición del arreglo: peso de la arista con respecto al valor de la "key"

def updateGraph():
    time = askTime()

    # map con pesos
    mapW = {}
    for key in map2:
        if key in latlong:
            k = getInterId(key)
            mapW[k] = []
        else:
            continue
        for arista in map2[key]:
            edge = (arista[0], arista[1])
            if edge in latlong:
                weight = calculateWeight(getOnlyLatLong(key), getOnlyLatLong(edge), time, arista[2])
                # [intersección, peso]
                mapW[k].append((getInterId(edge), weight))
        mapW[k]
    return mapW

In [65]:
def getMaxKey(G):
    return max(G, key = G.get)

In [ ]:
mapW = updateGraph()
printDictionary(mapW)

##**Shortest Paths**
Encuentra el camino más corto entre 2 puntos

In [55]:
def getMin(pq):
    weight = pq[0][1]
    index = 0
    # pair: (vértice, peso)
    for i, pair in enumerate(pq):
        if pair[1] < weight:
            weight = pair[1]
            index = i
    # index: pos en la pq que tiene el menor peso
    return index

getMin([(100, 124), (125, 10), (123, 100), (125, 127)])

1

In [69]:
# s: source node
# dijkstra
def findPaths(graph, s):
    n = getMaxKey(graph)
    
    path = [-1] * n
    visited = [False] * n
    
    cost = [math.inf] * n
    cost[s] = 0

    # vértice, peso 
    # el peso del 1er nodo siempre va a ser 0
    if graph[s]:
        priority_queue = [(s, 0)]
    else:
        print(f"No existe el vértice {s}")
        return

    #print(f"Tamaño del diccionario: {n}")
    #print(path)
    #print(visited)
    #print(cost)

    while priority_queue:
        # vértice, peso
        u, g = priority_queue.pop(getMin(priority_queue))
        if not visited[u]:
            visited[u] = True

        # loop over all the neighbours of the current node (s)
        # vértice, peso
        for v, w in graph[u]:
            # skip visited neighbours
            if not visited[v]:
                weight = g + w
                if weight < cost[v]:
                    cost[v] = weight
                    path[v] = u
                    priority_queue.append((v, weight))
    return path, cost

In [153]:
p, c = findPaths(mapW, 50)

In [ ]:
def definePath(camino):
    G = []
    for i in range(len(camino)):
        s = []
        for index, e in enumerate(camino):
            if e == i:
                s.append(index)
        G.append(s)
    return G

p = definePath(p)
print(p)

In [159]:
def bfs_al(G, n1, n2):
    n = len(G)
    visited = [False] * n
    path = [-1] * n
    queue = [ (n1, 0) ]
    visited[n1] = True

    while queue:
        # vértice, peso
        u, w = queue.pop(0)

        # si encuentra el punto final, retorna el peso del camino
        if u == n2: return path, w

        for v, p in G[u]:
            if visited[v] == False:
                visited[v] = True
                path[v] = u
                queue.append((v, w + p))
    return -1

path, weight = bfs_al(mapW, 30, 31)
path = definePath(path)

In [ ]:
def drawG_al(G, directed = False, path = []):
    graph = gv.Digraph("listaAdyacencia") if directed else gv.Graph("alegria")
    n = len(G)
    added = set()

    for v, u in enumerate(path):
        if u != -1:
            graph.edge(str(u), str(v), dir = "forward", penwidth = "2", color = "orange")
            added.add(f"{u}, {v}")
            added.add(f"{v}, {u}")
        
    for u in range(n):
        for v in G[u]:
            if not directed and not f"{u}, {v}" in added:
                added.add(f"{u}, {v}")
                added.add(f"{v}, {u}")
                graph.edge(str(u), str(v))
            elif directed:
                graph.edge(str(u), str(v))
    return graph

drawG_al(path)